## Configuration

In [124]:
# imports
import os
import sys
import time
import re
import glob
from math import pi
from datetime import datetime, timedelta
from dotmap import DotMap
import yaml
import hvplot.pandas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import plotly.graph_objects as go

import panel as pn
from panel.template import DarkTheme
from bokeh.io import output_file, show
from bokeh.models import (ColumnDataSource, HoverTool, Label, Legend, Span,
                          Div, LabelSet, FuncTickFormatter, CustomJS)
from bokeh.models.widgets import Panel, Tabs, Select
from bokeh.models.tickers import FixedTicker
from bokeh.plotting import figure
from bokeh.transform import dodge, cumsum
from bokeh.layouts import row, column, layout

In [125]:
def get_config():
    '''
    Function that gets the configuration of personalized variable definitions
    '''
    with open("config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config

In [126]:
def get_styling():
    css = '''
    :root {
        --mdc-theme-primary: rgba(0, 168, 65, 0.82) !important;
    }
    .bk-root .bk, .bk-root .bk:before, .bk-root .bk:after {
        font-family: Avenir Next, Helvetica !important;
    }
    .bk-root .slick-row.odd,
    .bk-root .slick-header-column:hover {
        background: rgba(0, 168, 65, 0.1) !important;
    }
    .bk-root .slick-cell.selected {
        background: rgba(0, 168, 65, 0.82) !important;
    }
    
    .bk-root .slick-sort-indicator-asc {
        background-image: url(data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIj8+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgeG1sbnM6c3ZnanM9Imh0dHA6Ly9zdmdqcy5jb20vc3ZnanMiIHZlcnNpb249IjEuMSIgd2lkdGg9IjUxMiIgaGVpZ2h0PSI1MTIiIHg9IjAiIHk9IjAiIHZpZXdCb3g9IjAgMCAyOTIuMzYyIDI5Mi4zNjIiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDUxMiA1MTIiIHhtbDpzcGFjZT0icHJlc2VydmUiIGNsYXNzPSIiPjxnPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgoJPHBhdGggZD0iTTI4Ni45MzUsMTk3LjI4NkwxNTkuMDI4LDY5LjM3OWMtMy42MTMtMy42MTctNy44OTUtNS40MjQtMTIuODQ3LTUuNDI0cy05LjIzMywxLjgwNy0xMi44NSw1LjQyNEw1LjQyNCwxOTcuMjg2ICAgQzEuODA3LDIwMC45LDAsMjA1LjE4NCwwLDIxMC4xMzJzMS44MDcsOS4yMzMsNS40MjQsMTIuODQ3YzMuNjIxLDMuNjE3LDcuOTAyLDUuNDI4LDEyLjg1LDUuNDI4aDI1NS44MTMgICBjNC45NDksMCw5LjIzMy0xLjgxMSwxMi44NDgtNS40MjhjMy42MTMtMy42MTMsNS40MjctNy44OTgsNS40MjctMTIuODQ3UzI5MC41NDgsMjAwLjksMjg2LjkzNSwxOTcuMjg2eiIgZmlsbD0iIzAwYTg0MSIgZGF0YS1vcmlnaW5hbD0iIzAwMDAwMCIgc3R5bGU9IiIvPgo8L2c+CjxnIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyI+CjwvZz4KPGcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPC9nPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgo8L2c+CjxnIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyI+CjwvZz4KPGcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPC9nPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgo8L2c+CjxnIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyI+CjwvZz4KPGcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPC9nPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgo8L2c+CjxnIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyI+CjwvZz4KPGcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPC9nPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgo8L2c+CjxnIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyI+CjwvZz4KPGcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPC9nPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgo8L2c+CjwvZz48L3N2Zz4K);
    }
    .bk-root .slick-sort-indicator-desc {
        background-image: url(data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIj8+CjxzdmcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgeG1sbnM6c3ZnanM9Imh0dHA6Ly9zdmdqcy5jb20vc3ZnanMiIHZlcnNpb249IjEuMSIgd2lkdGg9IjUxMiIgaGVpZ2h0PSI1MTIiIHg9IjAiIHk9IjAiIHZpZXdCb3g9IjAgMCAyOTIuMzYyIDI5Mi4zNjIiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDUxMiA1MTIiIHhtbDpzcGFjZT0icHJlc2VydmUiIGNsYXNzPSIiPjxnPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgoJPHBhdGggZD0iTTI4Ni45MzUsNjkuMzc3Yy0zLjYxNC0zLjYxNy03Ljg5OC01LjQyNC0xMi44NDgtNS40MjRIMTguMjc0Yy00Ljk1MiwwLTkuMjMzLDEuODA3LTEyLjg1LDUuNDI0ICAgQzEuODA3LDcyLjk5OCwwLDc3LjI3OSwwLDgyLjIyOGMwLDQuOTQ4LDEuODA3LDkuMjI5LDUuNDI0LDEyLjg0N2wxMjcuOTA3LDEyNy45MDdjMy42MjEsMy42MTcsNy45MDIsNS40MjgsMTIuODUsNS40MjggICBzOS4yMzMtMS44MTEsMTIuODQ3LTUuNDI4TDI4Ni45MzUsOTUuMDc0YzMuNjEzLTMuNjE3LDUuNDI3LTcuODk4LDUuNDI3LTEyLjg0N0MyOTIuMzYyLDc3LjI3OSwyOTAuNTQ4LDcyLjk5OCwyODYuOTM1LDY5LjM3N3oiIGZpbGw9IiMwMGE4NDEiIGRhdGEtb3JpZ2luYWw9IiMwMDAwMDAiIHN0eWxlPSIiIGNsYXNzPSIiLz4KPC9nPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgo8L2c+CjxnIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyI+CjwvZz4KPGcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPC9nPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgo8L2c+CjxnIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyI+CjwvZz4KPGcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPC9nPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgo8L2c+CjxnIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyI+CjwvZz4KPGcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPC9nPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgo8L2c+CjxnIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyI+CjwvZz4KPGcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPC9nPgo8ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciPgo8L2c+CjxnIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyI+CjwvZz4KPGcgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPC9nPgo8L2c+PC9zdmc+Cg==);
    }
    .bk-root .slick-sort-indicator-asc,
    .bk-root .slick-sort-indicator-desc {
        background-size: 10px 10px;
        background-repeat: no-repeat;
        margin-top: 3px;
        margin-right: 5px;
        width: 10px;
        height: 10px;
        float: right;
    }
    
    .bk-root .bk-tabs-header.bk-above .bk-headers-wrapper {
        border-bottom: none !important;
    }
    .bk-root .bk-tabs-header.bk-above .bk-tab {
    border-color: rgba(0, 168, 65, 0.3);
    margin-right: 3px;
    }
    .bk-root .bk-tabs-header .bk-tab.bk-active {
        border-color: rgba(0, 168, 65, 0.82) !important;
    }
    
    .main-content {
        overflow-x: scroll !important;
    }
    .modebar{
          display: none !important;
    }
    
    .selectDiv select {
      border-color: rgba(0, 168, 65, 0.82) !important;
      border-radius: 3px !important;
      background-image: url(data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLTg4NTktMSI/Pg0KPCEtLSBHZW5lcmF0b3I6IEFkb2JlIElsbHVzdHJhdG9yIDE2LjAuMCwgU1ZHIEV4cG9ydCBQbHVnLUluIC4gU1ZHIFZlcnNpb246IDYuMDAgQnVpbGQgMCkgIC0tPg0KPCFET0NUWVBFIHN2ZyBQVUJMSUMgIi0vL1czQy8vRFREIFNWRyAxLjEvL0VOIiAiaHR0cDovL3d3dy53My5vcmcvR3JhcGhpY3MvU1ZHLzEuMS9EVEQvc3ZnMTEuZHRkIj4NCjxzdmcgdmVyc2lvbj0iMS4xIiBpZD0iQ2FwYV8xIiB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHhtbG5zOnhsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIiB4PSIwcHgiIHk9IjBweCINCgkgd2lkdGg9IjI4NC45MjlweCIgaGVpZ2h0PSIyODQuOTI5cHgiIHZpZXdCb3g9IjAgMCAyODQuOTI5IDI4NC45MjkiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDI4NC45MjkgMjg0LjkyOTsiDQoJIHhtbDpzcGFjZT0icHJlc2VydmUiPg0KPGc+DQoJPHBhdGggZD0iTTI4Mi4wODIsNzYuNTExbC0xNC4yNzQtMTQuMjczYy0xLjkwMi0xLjkwNi00LjA5My0yLjg1Ni02LjU3LTIuODU2Yy0yLjQ3MSwwLTQuNjYxLDAuOTUtNi41NjMsMi44NTZMMTQyLjQ2NiwxNzQuNDQxDQoJCUwzMC4yNjIsNjIuMjQxYy0xLjkwMy0xLjkwNi00LjA5My0yLjg1Ni02LjU2Ny0yLjg1NmMtMi40NzUsMC00LjY2NSwwLjk1LTYuNTY3LDIuODU2TDIuODU2LDc2LjUxNUMwLjk1LDc4LjQxNywwLDgwLjYwNywwLDgzLjA4Mg0KCQljMCwyLjQ3MywwLjk1Myw0LjY2MywyLjg1Niw2LjU2NWwxMzMuMDQzLDEzMy4wNDZjMS45MDIsMS45MDMsNC4wOTMsMi44NTQsNi41NjcsMi44NTRzNC42NjEtMC45NTEsNi41NjItMi44NTRMMjgyLjA4Miw4OS42NDcNCgkJYzEuOTAyLTEuOTAzLDIuODQ3LTQuMDkzLDIuODQ3LTYuNTY1QzI4NC45MjksODAuNjA3LDI4My45ODQsNzguNDE3LDI4Mi4wODIsNzYuNTExeiIvPg0KPC9nPg0KPGc+DQo8L2c+DQo8Zz4NCjwvZz4NCjxnPg0KPC9nPg0KPGc+DQo8L2c+DQo8Zz4NCjwvZz4NCjxnPg0KPC9nPg0KPGc+DQo8L2c+DQo8Zz4NCjwvZz4NCjxnPg0KPC9nPg0KPGc+DQo8L2c+DQo8Zz4NCjwvZz4NCjxnPg0KPC9nPg0KPGc+DQo8L2c+DQo8Zz4NCjwvZz4NCjxnPg0KPC9nPg0KPC9zdmc+DQo=) !important;
      background-size: 10px 10px !important;
    }
    
    .selectDiv label {
        font-weight: bold;
    }
    .checkboxLabel {
        left: 10px !important;
    }
    
    .inputElement input[type="checkbox"] {
        width: 15px;
        height: 15px;
        border: 1px solid rgba(0, 168, 65, 0.82);
        border-radius: 2px;
        -webkit-appearance: none;
        -webkit-transition: box-shadow 200ms;
    }
    .inputElement input[type="checkbox"]:checked {
        background-color: rgba(0, 168, 65, 0.82);
    }
    
    .figureElement {
        box-shadow: 0px 1.5px 0px 0px rgba(0, 168, 65, 0.82);
        width: inherit !important;
    }
    
    .plotDescription p {
        text-align: justify;
    }
    '''
    return css

## Food intake

In [127]:
def preprocess_intake_files(self, subject):
    '''
    Author: Kylie Keijzer
    
    Function that preprocesses the intakes files from the given subject to the correct formattings
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    
    Returns
    --------------
    list
        baseline_filename    the filename of the created baseline intake file
        vegan_filename       the filename of the created vegan intake file
    '''
    
    days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    baseline = ["week1", "week2"]
    header = "Day,Date,Year,Calories,Fat,Saturated fat,Carbs,Fiber,Sugar,Protein,Sodium,Cholesterol,Potassium\n"
    
    food_intake_files_path = self.config['food_intake_files_path']
    processed_files_path = food_intake_files_path + '/processed'
    if not os.path.exists(processed_files_path):
        os.makedirs(processed_files_path)
    
    baseline_filename = processed_files_path + '/nutrition_baseline_subject_'+ subject +'.csv'
    nutrition_baseline = open(baseline_filename, 'w')
    nutrition_baseline.write(header)
    
    vegan_filename = processed_files_path + '/nutrition_vegan_subject_'+ subject +'.csv'
    nutrition_vegan = open(vegan_filename, 'w')
    nutrition_vegan.write(header)
    
    with nutrition_baseline as n_baseline, nutrition_vegan as n_vegan:
        for file in os.scandir(food_intake_files_path):
            filename = file.path
            # only use the file from the given subject
            if (('subject_' + subject in filename) and filename.lower().endswith('.csv')):
                file = open(file, 'r')
                with file as food_intake:
                    # start at line 10 with reading the food log
                    for i in range(9):
                        food_intake, next(food_intake)
                    for line in food_intake:
                        # replace double quotes with single quotes
                        index = 1 if '""' in line else 0
                        line = line.strip().replace('""', '"')[index:].rstrip('"')
                        # get the nutrition details per day
                        if line.split(',')[0].strip('"').lower() in days:
                            if any(week in filename.lower() for week in baseline):
                                n_baseline.write(line + '\n')
                            else:
                                n_vegan.write(line + '\n')
    
    return [baseline_filename, vegan_filename]

In [128]:
def create_nutrition_tables(self, subject):
    '''
    Author: Kylie Keijzer
    
    Function that creates the food intake data tables for the given subject
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    
    Returns
    --------------
    list
        nutrition_data    food intake data table
    '''
    
    nutrition_figures = []
    
    subject_data = preprocess_intake_files(self, subject)
    base_data = pd.read_csv(subject_data[0], sep=',')
    vegan_data = pd.read_csv(subject_data[1], sep=',')

    convert_columns = ['Calories', 'Fat', 'Saturated fat', 'Carbs', 
                       'Fiber', 'Sugar', 'Protein', 
                       'Sodium', 'Cholesterol', 'Potassium']

    # convert commas to points as delimiter and convert to numeric values
    base_data[convert_columns] = base_data[convert_columns].replace(',', '.', regex=True)
    base_data[convert_columns] = base_data[convert_columns].apply(pd.to_numeric, errors='coerce')
    base_data['Calories'] = base_data['Calories'].astype(float)
    vegan_data[convert_columns] = vegan_data[convert_columns].replace(',', '.', regex=True)
    vegan_data[convert_columns] = vegan_data[convert_columns].apply(pd.to_numeric, errors='coerce')
    vegan_data['Calories'] = vegan_data['Calories'].astype(float)
    
    # convert sodium, cholesterol and potassium to grams (they are initially in mg)
    base_data[convert_columns[7:]] = base_data[convert_columns[7:]] / 100
    vegan_data[convert_columns[7:]] = vegan_data[convert_columns[7:]] / 100
    
    # set the average saturated fat of the week to '0' if it is higher than the fat value
    base_data.loc[(base_data['Saturated fat'] > base_data['Fat']), 'Saturated fat'] = 0
    vegan_data.loc[(vegan_data['Saturated fat'] > vegan_data['Fat']), 'Saturated fat'] = 0
    
    # add the diet definition column
    base_data['Diet'] = 'Normal'
    vegan_data['Diet'] = 'Vegan'
    
    # concat diets and set date as index column
    nutrition_data = pd.concat([base_data, vegan_data])
    nutrition_data['Date'] = nutrition_data['Date'].astype(str) + ' ' + nutrition_data['Year'].astype(str)
    nutrition_data['Date'] = pd.to_datetime(nutrition_data['Date'], format=' %B %d %Y')
    nutrition_data.drop(['Year'], inplace=True, axis=1)
    nutrition_data.set_index('Date', inplace=True)
    nutrition_data.sort_index(inplace=True)
    
    return nutrition_data

In [129]:
def create_nutrition_boxplot(subject, nutrition_data):
    '''
    Author: Kylie Keijzer
    
    Function that creates the nutrition boxplot for the given subject
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    nutrition_data : DataFrame
        The food intake data table of the given subject
    
    Returns
    --------------
    figure
        The created nutrition boxplot visualization
    '''
    tabs = []
    # graph labels and positions
    plot_definitions = ['Calories', 'Macronutrients', 'Carbohydrates', 'Minerals']
    boxplot_data = get_labels_and_positions(plot_definitions)
    
    for group, data in zip(plot_definitions, boxplot_data):
        bar_labels = data[0]; base_positions = data[1]; vegan_positions = data[2]; label_positions = data[3]
        graph_labels = data[4]; outlier_plot_positions = data[5]; outlier_plot_positions_v = data[6]
    
        vegan = nutrition_data.loc[nutrition_data['Diet'] == 'Vegan'][bar_labels]
        baseline = nutrition_data.loc[nutrition_data['Diet'] == 'Normal'][bar_labels]

        # transform both dataframes to 2 column dataframe
        baseline_df = pd.DataFrame(columns=['group', 'value'])
        vegan_df = pd.DataFrame(columns=['group', 'value'])
        for label in bar_labels:
            tmp = pd.DataFrame(columns=['group', 'value'])
            tmp_v = pd.DataFrame(columns=['group', 'value'])
            tmp['value'] = baseline[label]
            tmp_v['value'] = vegan[label]
            tmp['group'] = label
            tmp_v['group'] = label
            baseline_df = pd.concat([baseline_df, tmp])
            vegan_df = pd.concat([vegan_df, tmp_v])
        
        # find the quartiles and IQR for each category
        groups = baseline_df.groupby('group', sort=False)
        groups_v = vegan_df.groupby('group', sort=False)
        q1 = groups.quantile(q=0.25)
        q1_v = groups_v.quantile(q=0.25)
        q2 = groups.quantile(q=0.5)
        q2_v = groups_v.quantile(q=0.5)
        q3 = groups.quantile(q=0.75)
        q3_v = groups_v.quantile(q=0.75)
        iqr = q3 - q1
        iqr_v = q3_v - q1_v
        upper = q3 + 1.5*iqr
        upper_v = q3_v + 1.5*iqr_v
        lower = q1 - 1.5*iqr
        lower_v = q1_v - 1.5*iqr_v

        # get the outliers
        out = groups.apply(get_outliers, upper=upper, lower=lower).dropna()
        out_v = groups_v.apply(get_outliers, upper=upper_v, lower=lower_v).dropna()
        
        # prepare outlier data for plotting
        if not out.empty:
            outx = []
            outy = []
            for keys in out.index:
                if isinstance(out, pd.DataFrame):
                    outx.append(keys)
                    outy.append(out.loc[keys].iloc[0])
                else:
                    outx.append(keys[0])
                    outy.append(out.loc[keys[0]].loc[keys[1]])
        if not out_v.empty:
            outx_v = []
            outy_v = []
            for keys in out_v.index:
                if isinstance(out_v, pd.DataFrame):
                    outx_v.append(keys)
                    outy_v.append(out_v.loc[keys].iloc[0])
                else:
                    outx_v.append(keys[0])
                    outy_v.append(out_v.loc[keys[0]].loc[keys[1]])

        if group == 'Calories':
            y_range = (0, 6000)
            title = "Boxplot representing caloric information of subject {}".format(subject)
            y_axis_label = "Calories (kcal)"
        else:
            y_range = (0, 500) if group == 'Macronutrients' else (0, 300)
            title = "Boxplot representing {} of subject {} ".format(group.lower(), subject)
            y_axis_label = "Grams (g)"
        
        p = figure(tools="", toolbar_location=None, 
                   plot_height=500, plot_width=500, y_range=y_range, title=title,
                   x_axis_label="Nutrient", y_axis_label=y_axis_label)

        # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
        qmin = groups.quantile(q=0.00)
        qmin_v = groups_v.quantile(q=0.00)
        qmax = groups.quantile(q=1.00)
        qmax_v = groups_v.quantile(q=1.00)
        upper.value = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'value']),upper.value)]    
        upper_v.value = [min([x,y]) for (x,y) in zip(list(qmax_v.loc[:,'value']),upper_v.value)]
        lower.value = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'value']),lower.value)]
        lower_v.value = [max([x,y]) for (x,y) in zip(list(qmin_v.loc[:,'value']),lower_v.value)]

        # stems
        p.segment(base_positions, upper.value, base_positions, q3.value, line_color="black")
        p.segment(base_positions, lower.value, base_positions, q1.value, line_color="black")
        p.segment(vegan_positions, upper_v.value, vegan_positions, q3_v.value, line_color="black")
        p.segment(vegan_positions, lower_v.value, vegan_positions, q1_v.value, line_color="black")

        # boxes
        p.vbar(base_positions, 0.4, q2.value, q3.value, fill_color="#008ae0", line_color="black", legend_label="Normal diet")
        p.vbar(base_positions, 0.4, q1.value, q2.value, fill_color="#008ae0", line_color="black")
        p.vbar(vegan_positions, 0.4, q2_v.value, q3_v.value, fill_color="#48BF91", line_color="black", legend_label="Vegan diet")
        p.vbar(vegan_positions, 0.4, q1_v.value, q2_v.value, fill_color="#48BF91", line_color="black")

        # whiskers
        p.rect(base_positions, lower.value, 0.2, 0.01, line_color="black")
        p.rect(base_positions, upper.value, 0.2, 0.01, line_color="black")
        p.rect(vegan_positions, lower_v.value, 0.2, 0.01, line_color="black")
        p.rect(vegan_positions, upper_v.value, 0.2, 0.01, line_color="black")

        # outliers
        if not out.empty:
            p.circle([outlier_plot_positions[x] for x in outx], outy, size=3, color="#F38630", fill_alpha=0.6)
        if not out_v.empty:
            p.circle([outlier_plot_positions_v[x] for x in outx_v], outy_v, size=3, color="#F38630", fill_alpha=0.6)

        p.xgrid.grid_line_color = None
        p.xaxis.major_label_text_font_size = "10px"
        p.xaxis.major_label_orientation = 0.5
        p.xaxis.ticker = label_positions

        # set labels at the tick positions
        p.xaxis.formatter = FuncTickFormatter(code="""
                var mapping = {};
                return mapping[tick];
            """.format(graph_labels))
        
        tab = Panel(child=p, title=group)
        tabs.append(tab)
    
    tabs = Tabs(tabs=tabs)
    return tabs

def get_labels_and_positions(columns):
    '''
    Author: Kylie Keijzer
    
    Function that creates the labels and position of the different nutrition boxplots
    '''
    
    boxplot_data = []
    for label in columns:
        if label == 'Calories':
            bar_labels = ['Calories']
            base_positions = [0.5]
        elif label == 'Macronutrients':
            bar_labels = ['Carbs', 'Protein', 'Fat', 'Saturated fat']
            base_positions = [0.5, 2.5, 4.5, 6.5]
        elif label == 'Carbohydrates':
            bar_labels = ['Fiber', 'Sugar']
            base_positions = [0.5, 2.5]
        elif label == 'Minerals':
            bar_labels = ['Sodium', 'Cholesterol', 'Potassium']
            base_positions = [0.5, 2.5, 4.5]
        
        vegan_positions = [pos+0.6 for pos in base_positions]
        label_positions = [pos+0.3 for pos in base_positions]
        graph_labels = str(dict(zip(label_positions, bar_labels)))
        outlier_plot_positions = dict(zip(bar_labels, base_positions))
        outlier_plot_positions_v = dict(zip(bar_labels, vegan_positions))
        boxplot_data.append([bar_labels, base_positions, vegan_positions, label_positions, 
                                    graph_labels, outlier_plot_positions, outlier_plot_positions_v])
    return boxplot_data

def get_outliers(group, upper, lower):
    '''
    Author: Kylie Keijzer
    
    Function that finds the outliers for each category
    '''
    
    cat = group.name
    return group[(group.value > upper.loc[cat]['value']) | (group.value < lower.loc[cat]['value'])]['value']

In [130]:
def create_average_nutrition_figure(subject, nutrition_data):
    '''
    Author: Kylie Keijzer
    
    Function that creates the averages nutrition graph for the given subject
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    nutrition_data : DataFrame
        The food intake data table of the given subject
    
    Returns
    --------------
    figure
        The created nutrition graph visualization
    '''
    
    # calcuate the average nutritional values
    averages = nutrition_data.groupby(['Diet']).mean().round(2)
    
    bar_labels = ['Carbs', 'Protein', 'Fat', 
                 'Saturated fat', 'Fiber', 'Sugar', 
                 'Sodium', 'Cholesterol', 'Potassium']
    data_obj = {
            'labels': bar_labels,
            'Normal': averages[bar_labels].loc['Normal'],
            'Vegan': averages[bar_labels].loc['Vegan'],
            'None' : [0 for i in bar_labels]
        }
    source = ColumnDataSource(data=data_obj)
    
    p = figure(x_range=bar_labels, y_range=(0, 400), plot_height=500, plot_width=700,
               title="Average nutritional information subject " + subject + " in grams",
               x_axis_label="Nutrient", y_axis_label="Grams (g)", 
               toolbar_location=None, tools="")
    
    p.vbar(x=dodge('labels', -0.12, range=p.x_range), 
           top='Normal', name='Normal', width=0.2, 
           source=source, color="#008ae0", legend_label="Normal diet", line_color="#007ecc")

    # middle label
    p.vbar(x=dodge('labels',  0,  range=p.x_range), top='None', width=0.1, source=source)
    
    p.vbar(x=dodge('labels',  0.12,  range=p.x_range), 
           top='Vegan', name='Vegan', width=0.2, 
           source=source, color="#48BF91", legend_label="Vegan diet", line_color="#3eb185")
    
    p.x_range.range_padding = 0.05
    p.xgrid.grid_line_color = None
    p.legend.location = "top_right"
    p.legend.orientation = "vertical"
    p.xaxis.major_label_orientation = 0.5
    p.xaxis.major_label_text_font_size = "10px"
    
    p.add_tools(HoverTool(
        names = ['Normal', 'Vegan'],
        tooltips = [
            ('', '$name diet'),
            ('', '@$name{1.11} grams')
        ],
        mode = 'mouse',
        show_arrow = False,
        point_policy = 'follow_mouse'
    ))
    
    style = {
        'font-size': 'smaller',
        'margin-top': '20px'
    }
    div = Div(text="""
        <p><b>Average calories:</b><br>
        Normal diet: {} kcal <br>
        Vegan diet:  {} kcal</p>
        """.format(averages['Calories'].loc['Normal'], averages['Calories'].loc['Vegan']),
              width=200, height=100, style=style)
    
    return pn.Row(p, div)

In [131]:
def create_nutrition_piecharts(subject, nutrition_data):
    '''
    Author: Kylie Keijzer
    
    Function that creates the food intake pie charts per week for the given subject
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    nutrition_data : list
        The food intake data table
    
    Returns
    --------------
    figure
        The tab-seperated intake pie charts
    '''
    
    tabs = []
    weekly_averages = get_weekly_averages(nutrition_data)
    week_starts = {0: '2020-10-12', 1: '2020-10-19', 2: '2020-10-26', 
                   3: '2020-11-02', 4: '2020-11-09', 5: '2020-11-16'}
    
    macro_colors = ['#266298', '#89c609', '#fc2a35']
    sug_fib_colors = ['#f9dda9', '#ac545c']
    micro_colors = ['#a21d22', '#f6871e']
    
    for i in range(0, len(weekly_averages.index)):
        macros = weekly_averages[['Carbs', 'Protein', 'Fat']].iloc[i]
        sug_fib = weekly_averages[['Sugar', 'Fiber']].iloc[i]
        micros = weekly_averages[['Sodium', 'Potassium']].iloc[i]

        p1 = get_piechart_data(macros, macro_colors, 
                               'Average calorie composition subject {}'.format(subject))
        p2 = get_piechart_data(sug_fib, sug_fib_colors, 
                               'Average sugar and fiber devision subject {}'.format(subject))
        p3 = get_piechart_data(micros, micro_colors, 
                               'Average sodium and potassium devision subject {}'.format(subject))

        pie_charts = row(p1, p2, p3)
        tab = Panel(child=pie_charts, title="Week " + str(i+1))
        tabs.append(tab)
    
    tabs = Tabs(tabs=tabs)
    return tabs

In [132]:
def get_piechart_data(nutrients, nutrient_colors, chart_description):
    '''
    Author: Kylie Keijzer
    
    Function that creates a food intake pie chart for the given nutrients DataFrame
    
    Parameters
    --------------
    nutrients : DataFrame
        The DataFrame with the nutrient values
    nutrition_colors : list
        The nutrient colors list
    chart_description : string
        The title of the pie chart graph
    
    Returns
    --------------
    p
        The generated pie chart
    '''
    
    R = 0.35
    p_range = (-R * 1.1, R * 1.5)
    p_size = 350
    
    data = (pd.Series(nutrients.to_dict())
            .reset_index(name='value')
            .rename(columns={'index':'nutrient'})
            .assign(end_angle=lambda d: np.cumsum(d['value'] / d['value'].sum() * 2 * pi),
                    start_angle=lambda d: np.pad(d['end_angle'], (1, 0), mode='constant')[:-1],
                    label_x=lambda d: R * 0.9 * np.cos(d['start_angle']),
                    label_y=lambda d: R * 0.95 * np.sin(d['start_angle'])))
    data['percentage'] = (data['value']/data['value'].sum() * 100).round(2).astype(str) + '%'
    data['color'] = nutrient_colors[:len(nutrients)]

    p = figure(title=chart_description, toolbar_location=None,
               plot_height=p_size, plot_width=p_size,
               x_range=p_range, y_range=p_range,
               tools="")
    p.wedge(x=0, y=0, radius=R,
            start_angle='start_angle', end_angle='end_angle',
            line_color="white", fill_color='color', legend_field='nutrient', source=data)

    source = ColumnDataSource(data)
    labels = LabelSet(x='label_x', y='label_y', text='percentage',
                      angle='start_angle', source=source, render_mode='canvas', 
                      text_align='right', text_font_size="9pt", text_color='white')

    p.add_layout(labels)
    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None
    p.legend.label_text_font_size = '9pt'
    
    return p

In [133]:
def get_weekly_averages(nutrition_data):
    '''
    Author: Kylie Keijzer
    
    Function that calculates the weekly average nutrient values
    '''
    
    nutrition_data['week_number'] = ((nutrition_data.Day.str.lower() == 'monday').cumsum())
    return nutrition_data.groupby('week_number').mean().round(2)

In [134]:
def create_nutrition_dataframe(nutrition_data):
    '''
    Author: Kylie Keijzer
    
    Function creates the 'DOM' intake dataframe for visual representation
    
    Parameters
    --------------
    nutrition_data : DataFrame
        The given DataFrame of the subject
    
    Returns
    --------------
    panel DataFrame
        The DataFrame widget created for visual representation
    '''
    
    column_order = ['Carbs', 'Protein', 'Fat', 
                 'Saturated fat', 'Fiber', 'Sugar', 
                 'Sodium', 'Cholesterol', 'Potassium']
    
    weekly_df = nutrition_data.drop(columns=['Diet', 'week_number'])
    # add the mean and the median
    weekly_df.index = weekly_df.index.astype(str)
    weekly_df = pd.concat([weekly_df.round(2), weekly_df.mean().round(2).to_frame('Mean').T,
                        weekly_df.median().round(2).to_frame('Median').T])
    weekly_df.fillna('', inplace=True)
    weekly_df = pd.concat([weekly_df.iloc[:, 0:2], weekly_df[column_order].add_suffix(' (g)')], axis=1)
    
    descr = Div(text="""
        <p><b>Food intake table</b><br>
        This food intake table represents the consumed nutrients of the selected subject per day. The two bottom 
        rows represent the mean and the median of the different nutrients. Table rows can be sorted by clicking 
        the column you want to sort on. </p>
    """, width=600, height=100, css_classes=['plotDescription'])
    data_frame = pn.widgets.DataFrame(weekly_df, autosize_mode='fit_columns', frozen_rows=-2,
                                width=1050, height=350)
    
    return pn.Column(descr, data_frame)

# Microbiota

In [135]:
def load_data_microbiota(PATH):
    """
    Author: Tijs van Lieshout
    Load microbiota data (Gut Feeling Knowledge Base and metaphlann output) into pandas 
    dataframes

    Keyword arguments:
    PATH -- The path which contains the Gut Feeling Knowledge Base and metaphlann output dir
    
    Returns:
    gfkb -- A pandas dataframe containing the Gut Feeling Knowledge Base
    tax_profiles -- A pandas dataframe containing the taxonomic profile per barcode for all barcodes"""
    # Gut Feeling Knowledge Base
    gfkb = pd.read_csv(f"{PATH}/GutFeelingKnowledgeBase-v4-Master_List.csv")
    gfkb = gfkb.drop(columns=["Present in GFKB v3 (Y/N)",
                              "Present in GFKB_epilepsy v3 (Y/N)"])
    gfkb = pd.concat([gfkb.drop(columns=["Genome Size (Mb)"]).apply(lambda x: x.astype(str)), 
                      gfkb["Genome Size (Mb)"]], axis=1)

    tax_profiles = pd.DataFrame()
    
    # Dictionary containing barcode ID as key and a list with subject ID and bool if vegan as values
    barcode2subject_sample = {"barcode_01":["A", True],
                              "barcode_02":["B", True],
                              "barcode_03":["C", True],
                              "barcode_04":["D", True],
                              "barcode_05":["E", True],
                              "barcode_06":["A", False],
                              "barcode_07":["B", False],
                              "barcode_08":["C", False],
                              "barcode_09":["D", False],
                              "barcode_10":["E", False]}
    
    # Concatenate all taxonomic profiles of all barcodes to one dataframe
    tax_profiles = concat_tax_profiles(PATH, tax_profiles, barcode2subject_sample)
    
    return gfkb, tax_profiles

In [136]:
def concat_tax_profiles(PATH, tax_profiles, barcode2subject_sample):
    """
    Author: Tijs van Lieshout
    Concatenate all taxonomic profiles of all given barcodes in the metaphlan output dir
    to one dataframe and reset the index on multi-index (subject, is_vegan).

    Keyword arguments:
    PATH -- The path which contains the Gut Feeling Knowledge Base and metaphlann output dir
    tax_profiles -- An empty pandas dataframe containing the column names in which all taxonomic
    profiles will be concatenated.
    barcode2subject_sample -- Dictionary containing barcode ID as key and a list with subject ID 
    and bool if sample is vegan as values
    
    Returns:
    tax_profiles -- A pandas dataframe containing the taxonomic profile per barcode for all barcodes"""
    for file in glob.glob(os.path.abspath(f"{PATH}/adj_align_output_relaxed/*.txt")):
        tax_profile = pd.read_csv(file, 
                                  comment="#", 
                                  sep="\t", 
                                  names=["clade_name", 
                                         "NCBI_tax_id", 
                                         "relative_abundance",
                                         "additional_species"])
        # Splitting clade_name into taxonomic levels
        tax_profile = tax_profile.join(tax_profile["clade_name"].str.split('|', expand=True).rename(columns={0:'kingdom', 
                                                                                                             1:'phylum', 
                                                                                                             2:'class', 
                                                                                                             3:'order', 
                                                                                                             4:'family', 
                                                                                                             5:'genus', 
                                                                                                             6:'species'}), how='left')
        # Indexing
        barcode = file.split('adj_align_output_relaxed/')[1].split("_all")[0]
        tax_profile["subject"] = barcode2subject_sample[barcode][0]
        tax_profile["is_vegan"] = barcode2subject_sample[barcode][1]
        tax_profile = tax_profile.set_index([tax_profile.subject, tax_profile.is_vegan]).sort_index()
        
        tax_profiles = pd.concat([tax_profiles,
                                  tax_profile])
    # Clean up of dataframe
    
    return cleanup_taxonomic_profiles(tax_profiles)

In [137]:
def cleanup_taxonomic_profiles(tax_profiles):
    """
    Author: Tijs van Lieshout
    cleanup taxonomic profiles based on stripping of metaphlan artifacts.

    Keyword arguments:
    tax_profiles -- all taxonomic profiles from metaphlan
    
    Returns:
    tax_profiles -- A pandas dataframe containing the taxonomic profile per barcode for all barcodes sorted"""
    tax_profiles["kingdom"] = tax_profiles["kingdom"].str.strip("k__")
    tax_profiles["phylum"] = tax_profiles["phylum"].str.strip("p__")
    tax_profiles["class"] = tax_profiles["class"].str.strip("c__")
    tax_profiles["order"] = tax_profiles["order"].str.strip("o__")
    tax_profiles["family"] = tax_profiles["family"].str.strip("f__")
    tax_profiles["genus"] = tax_profiles["genus"].str.strip("g__")
    tax_profiles["species"] = tax_profiles["species"].str.strip("s__")
    tax_profiles = tax_profiles.drop(columns=["clade_name"])
    tax_profiles = tax_profiles[["kingdom", 
                                 "phylum", 
                                 "class", 
                                 "order", 
                                 "family", 
                                 "genus", 
                                 "species", 
                                 "relative_abundance", 
                                 "NCBI_tax_id", 
                                 "additional_species"]]
    return tax_profiles.sort_index()

In [138]:
def create_column_data_source(taxa_abundance, taxa, add_diff = False):
    """
    Author: Tijs van Lieshout
    Create a Bokeh ColumnDataSource for the plotting of taxa_abundance data for the taxa in the taxa list.
    
    Keyword arguments:
    taxa_abundance -- A pandas dataframe containing the abundance of taxa for the vegan and regular diet samples for a subject
    taxa -- A list of taxa found in the taxa_abundance columns
    add_diff -- A Boolean which results in some additional calculations for the ColumnDataSource used by the dumbbell plot
    
    Returns:
    A bokeh ColumnDataSource containing the taxa, abundance of taxa in control, abundance of taxa in vegan and None for labels
    """
    diet = ['control', 'vegan']

    # Taking the mean of a single data point just for bokeh to accept the format of the ColumnDataSource
    grouped = taxa_abundance.groupby(['is_vegan']).mean()
    
    # Handling if there are either no datapoints before or after the vegan intervention
    if False in grouped.index:
        control = grouped[taxa].loc[False]
    else:
        control = [np.nan for taxon in taxa]
    if True in grouped.index:
        vegan = grouped[taxa].loc[True]
    else:
        vegan = [np.nan for taxon in taxa]
    if add_diff and True in grouped.index and False in grouped.index:
        diff = pd.concat([control, vegan], axis=1, sort=False)
        diff = diff.rename(columns = {False:'control', True:'vegan'}).sort_values(by=['control', 'vegan'], ascending=True, na_position='first')
        xcontrol = diff['control']
        xvegan = diff['vegan']
        data = {'taxa': list(diff.index),
                'control': xcontrol,
                'vegan': xvegan,}
    else:
        data = {'taxa': taxa,
                'control': control,
                'vegan': vegan,
                'None': [0 for i in taxa]}

    return ColumnDataSource(data=data)

In [139]:
def recreate_zimmer(tax_profiles, subject):
    """
    Author: Tijs van Lieshout
    Recreate a comparison of taxa that have been routinely analysed by Zimmer et al. 2012
    
    Keyword arguments:
    tax_profiles -- A pandas dataframe containing the taxonomic profile
    subject -- A string consisting of the subject ID
    
    Returns:
    plot_zimmer() -- The plotting function of the barplot of taxa deemed interesting by Zimmer et al. 2012
    """
    
    
    if subject == "s Pooled":
        tax_profiles_subject = tax_profiles
    else:
        tax_profiles_subject = tax_profiles[tax_profiles.index.get_level_values('subject') == subject]
        subject = f" {subject}"
    
    # Subset for the Zimmer et al. 2012 bar plot
    bacteroides_subset = tax_profiles_subject[(tax_profiles_subject['genus'] == "Bacteroides") & (tax_profiles_subject['species'].isnull())]
    bifidobacteria_subset = tax_profiles_subject[(tax_profiles_subject['genus'] == "Bifidobacterium") & (tax_profiles_subject['species'].isnull())]
    ecoli_subset = tax_profiles_subject[(tax_profiles_subject['species'] == "Escherichia_coli")]
    enterobacter_subset = tax_profiles_subject[(tax_profiles_subject['family'] == "Enterobacteriaceae") & (tax_profiles_subject['genus'].isnull())]

    # Other taxa Zimmer et al. 2012 deemed of interest
    clostridia_subset = tax_profiles_subject[(tax_profiles_subject['class'] == "Clostridia") & (tax_profiles_subject['order'].isnull())]
    
    zimmer_subset = pd.concat([bacteroides_subset,
                               bifidobacteria_subset,
                               ecoli_subset,
                               enterobacter_subset,
                               clostridia_subset]).sort_index()
    
    
    return plot_zimmer(zimmer_subset, subject)

In [140]:
def plot_zimmer(zimmer_subset, subject):
    """
    Author: Tijs van Lieshout
    Plot the taxa deemed interesting by Zimmer et al. 2012 for a subject as a grouped barplot. Based on code by Kylie Keijzer
    
    Keyword arguments:
    zimmer_subset -- A pandas dataframe containing a subset of taxa of interest of the taxonomic profile
    dataframe containing only the taxa analyzed by zimmer et al. 2012 as values
    subject -- A string consisting of the subject ID
    
    Returns: 
    A tab layout containing the grouped bar plot of the taxa deemed interesting by Zimmer et al. 2012 for a subject
    """
    if not zimmer_subset.empty:
        tabs = []
        taxa = ['Bacteroides', 'Bifidobacterium', 'Escherichia_coli', 'Enterobacteriaceae', 'Clostridia']
        taxa_abundance = create_taxa_abundance_zimmer(zimmer_subset)
        if not taxa_abundance.empty:
            
            source = create_column_data_source(taxa_abundance, taxa)
            
            # Creating the figure
            p = figure(x_range=taxa, y_range=(0, 100), plot_height=500, plot_width=800,
               title=f"Subject{subject}: Abundance of specific taxa of interest",
               x_axis_label="Taxon", y_axis_label="Relative abundance (%)", 
               toolbar_location=None, tools="")

            p.vbar(x=dodge('taxa', -0.12, range=p.x_range), 
                   top='control', name='control', width=0.2, 
                   source=source, color="#008ae0", legend_label="Regular diet", line_color="#007ecc")

            # middle label
            p.vbar(x=dodge('taxa',  0,  range=p.x_range), top='None', width=0.1, source=source)

            p.vbar(x=dodge('taxa',  0.12,  range=p.x_range), 
                   top='vegan', name='vegan', width=0.2, 
                   source=source, color="#48BF91", legend_label="Vegan diet", line_color="#3eb185")

            p.x_range.range_padding = 0.1
            p.xgrid.grid_line_color = None
            p.legend.location = "top_right"
            p.legend.orientation = "horizontal"

            tab = Panel(child=p, title="Taxa of interest")
            tabs.append(tab)

            tabs = Tabs(tabs=tabs)
            return tabs

In [141]:
def create_taxa_abundance_zimmer(zimmer_subset):
    """
    Author: Tijs van Lieshout
    Plot the taxa deemed interesting by Zimmer et al. 2012 for a subject as a grouped barplot.
    
    Keyword arguments:
    zimmer_subset -- A pandas dataframe containing a subset of taxa of interest of the taxonomic profile
    dataframe containing only the taxa analyzed by zimmer et al. 2012 as values
    
    Returns: 
    A pandas dataframe containing the relative abundance per taxa per sample/barcode
    """
    taxa_abundance = pd.DataFrame()
    # Subset for the Zimmer et al. 2012 bar plot
    taxa_abundance['Bacteroides'] = zimmer_subset[(zimmer_subset['genus'] == "Bacteroides") & 
                                                  (zimmer_subset['species'].isnull())]['relative_abundance']
    taxa_abundance['Bifidobacterium'] = zimmer_subset[(zimmer_subset['genus'] == "Bifidobacterium") & 
                                                      (zimmer_subset['species'].isnull())]['relative_abundance']
    taxa_abundance['Escherichia_coli'] = zimmer_subset[(zimmer_subset['species'] == "Escherichia_coli")]['relative_abundance']
    taxa_abundance['Enterobacteriaceae'] = zimmer_subset[(zimmer_subset['family'] == "Enterobacteriaceae") & 
                                                         (zimmer_subset['genus'].isnull())]['relative_abundance']
    # Other taxa Zimmer et al. 2012 deemed of interest
    taxa_abundance['Clostridia'] = zimmer_subset[(zimmer_subset['class'] == "Clostridia") & 
                                                 (zimmer_subset['order'].isnull())]['relative_abundance']
    return taxa_abundance

In [142]:
def select_taxa_on_level(taxa_abundance, tax_profiles_subject, tax_level):
    """
    Author: Tijs van Lieshout
    Create a subset of a pandas dataframe containing the abundance of taxa for the vegan and regular diet samples for a subject
    based on the taxonomic level selected
    
    Keyword arguments:
    taxa_abundance -- A pandas dataframe containing the abundance of taxa for the vegan and regular diet samples for a subject
    tax_profiles_subject -- A pandas dataframe containing the taxonomic profile for the vegan and regular diet samples for a subject
    tax_level -- A string containing the taxonomic level to select on
    
    Returns:
    taxa_abundance -- A pandas dataframe containing the abundance of taxa for the vegan and regular diet samples for a subject, 
    now selected for taxonomic level
    taxa -- A list containing the unique taxa found in the taxa_abundance dataframe
    """
    if tax_level == "kingdom":
        not_included = "phylum"
    elif tax_level == "phylum":
        not_included = "class"
    elif tax_level == "class":
        not_included = "order"
    elif tax_level == "order":
        not_included = "family"
    elif tax_level == "family":
        not_included = "genus"
    elif tax_level == "genus":
        not_included = "species"
    elif tax_level == "species":
        taxa = list(tax_profiles_subject[(tax_profiles_subject[tax_level].notnull())][tax_level])
        for taxon in taxa:
            taxa_abundance[taxon] = tax_profiles_subject[(tax_profiles_subject[tax_level] == taxon)]['relative_abundance']
        return taxa_abundance, list(set(taxa))
    else:
        raise Exception("incorrect taxonomic level selected")
    
    taxa = list(tax_profiles_subject[(tax_profiles_subject[tax_level].notnull()) & 
                                     (tax_profiles_subject[not_included].isnull())][tax_level])
    for taxon in taxa:
        taxa_abundance[taxon] = tax_profiles_subject[(tax_profiles_subject[tax_level] == taxon) & 
                                                     (tax_profiles_subject[not_included].isnull())]['relative_abundance']
    return taxa_abundance, list(set(taxa))

In [143]:
def plot_dumbbell(tax_profiles, subject):
    """
    Author: Tijs van Lieshout
    Plot the zimmer subset per subject as a grouped barplot. Based on code by Kylie Keijzer
    
    Keyword arguments:
    zimmer_subset -- A pandas dataframe containing a subset of taxa of interest of the taxonomic profile per barcode for all barcodes 
    dataframes containing only the taxa analyzed by zimmer et al. 2012 as values
    subject -- A string consisting of the subject ID
    
    Returns: 
    A tab layout containing the dumbbell plot of all taxa of a given taxonomic level for a subject
    """
    tax_levels = ['kingdom','phylum','class','order','family','genus','species']
    
    tabs = []
    if not tax_profiles.empty:
        for tax_level in tax_levels:
            title = subject
            taxa_abundance = pd.DataFrame()
            if subject == "s Pooled":
                tax_profiles_subject = tax_profiles
            else:
                tax_profiles_subject = tax_profiles[tax_profiles.index.get_level_values('subject') == subject]
                title = f" {subject}"

            taxa_abundance, taxa = select_taxa_on_level(taxa_abundance, tax_profiles_subject, tax_level)

            if not taxa_abundance.empty:

                source = create_column_data_source(taxa_abundance, taxa, True)
                # Get dataframe based on order from source
                source_df = source.to_df()
                taxa_sorted = list(source_df.index)

                n_taxa = len(taxa_sorted)

                if n_taxa < 10:
                    n_taxa = 10
                
                # Creating the figure
                p = figure(y_range=taxa_sorted, plot_height=25*n_taxa, 
                           plot_width=1200, title=f"Subject{title}: Abundance of all taxa on {tax_level} level",
                           y_axis_label="Taxon", x_axis_label="Relative abundance (%)", 
                           toolbar_location=None, tools="")

                if 'xvegan' or 'xregular' in source_df.columns:
                    p.segment(x0="control", y0=(dodge('taxa', 0, range=p.y_range)), x1="vegan", 
                              y1=dodge('taxa', 0, range=p.y_range), line_color="black", line_width=3, source=source)

                p.circle(y=dodge('taxa', 0, range=p.y_range), x='control', name='control',
                        source=source, color="#008ae0", legend_label="Regular diet", size=12)

                p.circle(y=dodge('taxa', 0, range=p.y_range), x='vegan', name='vegan',
                        source=source, color="#48BF91", legend_label="Vegan diet", size=12)
                
                p.add_tools(HoverTool(
                    names = ['control', 'vegan'],
                    tooltips = [
                        ('', '$name'),
                        ('', '@$name{1.11}%')
                    ],
                    mode = 'mouse',
                    show_arrow = False,
                    point_policy = 'follow_mouse'
                ))

                p.y_range.range_padding = 0.1
                p.ygrid.grid_line_color = None 
                p.legend.location = "bottom_right"
                p.legend.orientation = "vertical"
                tab = Panel(child=p, title=tax_level)
                tabs.append(tab)

        tabs = Tabs(tabs=tabs, sizing_mode="scale_height")
        return tabs

## Body composition

In [144]:
def create_body_comp_plots(self, sub):
    '''
    Author: Behzad Barati
    
    '''
    
    filepath = self.config['body_comp_files_path']
    df = pd.read_excel(filepath + '/df_BC.xlsx')
    self.body_comp_dataframe = df.copy()
    dft = df[df['Subject'] == sub]
    line_plots = pn.Tabs(
            ('Weight' , dft.hvplot.scatter(x='Date', y='Bodyweight (kg)', by='vegan', cmap=['#008ae0', '#48BF91'], s=100, height=400, width=800)
             *dft.hvplot.line(x='Date', y='Bodyweight (kg)', color='black')),
            ('BMI',     dft.hvplot.scatter(x= 'Date', y='BMI (score)', by='vegan', cmap=['#008ae0', '#48BF91'], s=100, height=400, width=800)
             *dft.hvplot.line(x='Date', y='BMI (score)', color='black')),
            ('Bodyfat', dft.hvplot.scatter(x='Date', y='Bodyfat (%)', by='vegan', cmap=['#008ae0', '#48BF91'], s=100, height=400, width=800)
             *dft.hvplot.line(x='Date', y='Bodyfat (%)', color='black')),
            ('Muscle',  dft.hvplot.scatter(x='Date', y='Skeletal muscle (%)', by='vegan', cmap=['#008ae0', '#48BF91'], s=100, height=400, width=800)
             *dft.hvplot.line(x='Date', y='Skeletal muscle (%)', color='black')))
    boxplots = pn.Tabs(
            ('Weight' , dft.hvplot.box(y='Bodyweight (kg)', by='vegan', color='vegan', cmap=['#008ae0', '#48BF91'], legend=False, height=400, width=800)),
            ('BMI',     dft.hvplot.box(y='BMI (score)', by='vegan', color='vegan', cmap=['#008ae0', '#48BF91'], legend=False, height=400, width=800)),
            ('Bodyfat', dft.hvplot.box(y='Bodyfat (%)', by='vegan', color='vegan', cmap=['#008ae0', '#48BF91'], legend=False, height=400, width=800)),
            ('Muscle',  dft.hvplot.box(y='Skeletal muscle (%)', by='vegan', color='vegan', cmap=['#008ae0', '#48BF91'], legend=False, height=400, width=800)))
    violins = pn.Tabs(
            ('Weight' , dft.hvplot.violin(y='Bodyweight (kg)', by='vegan' ,padding=0.5, color='vegan', cmap=['#008ae0', '#48BF91'], height=550, width=800)),
            ('BMI',     dft.hvplot.violin(y='BMI (score)', by='vegan' ,padding=0.5, color='vegan', cmap=['#008ae0', '#48BF91'], height=550, width=800)),
            ('Bodyfat', dft.hvplot.violin(y='Bodyfat (%)', by='vegan' ,padding=0.5, color='vegan', cmap=['#008ae0', '#48BF91'], height=550, width=800)),
            ('Muscle',  dft.hvplot.violin(y='Skeletal muscle (%)', by='vegan' ,padding=0.5, color='vegan', cmap=['#008ae0', '#48BF91'], height=550, width=800)))
    
    return [line_plots, boxplots, violins]

In [145]:
def create_body_comp_tables(self, subject):
    '''
    Author: Kylie Keijzer
    
    Function creates the 'DOM' body composition dataframe for visual representation
    
    Parameters
    --------------
    subject : char
        The given subject
    
    Returns
    --------------
    panel DataFrame
        The DataFrame widget created for visual representation
    '''
    
    temp_df = self.body_comp_dataframe.copy()
    temp_df = temp_df[temp_df['Subject'] == subject]
    temp_df.drop(temp_df.columns[[0, 1]], axis=1, inplace=True)
    temp_df.set_index('Date', inplace=True)
    temp_df.index = temp_df.index.astype(str)
    temp_df.rename(columns={'vegan': 'Vegan diet'}, inplace=True)
    temp_df.replace({'Vegan diet': {True: 'True', False: 'False'}}, inplace = True)
    
    # add the mean and the median
    temp_df = pd.concat([temp_df.round(2), temp_df.mean().round(2).to_frame('Mean').T,
                        temp_df.median().round(2).to_frame('Median').T])
    temp_df.fillna('', inplace=True)
    
    descr = Div(text="""
        <p><b>Body composition measurements table</b><br>
        This body composition measurements table represents the measured variables of the selected subject per day. 
        The two bottom rows represent the mean and the median of the different variables. Table rows can be sorted 
        by clicking the column you want to sort on. </p>
    """, width=600, height=100, css_classes=['plotDescription'])
    data_frame = pn.widgets.DataFrame(temp_df, autosize_mode='fit_columns', frozen_rows=-2,
                                width=1300, height=350)
    
    return pn.Column(descr, data_frame)

## Mental health data & plots

In [146]:
def load_mental_files(subject):
    """
    Author: Henrike Vaartstra
    
    Function loads the preprocessed files for the given subject; the pre-processing can be found in 
    'preprocessing_mental_health' within the henrike branch in git repo
    
    Parameters
        subject:            The subject's initial
    
    Returns
        mental_data:          The dataframe with the data for the baseline and vegan weeks
        baseline_data:        The dataframe with the data for the vegan weeks
        vegan_data:           The dataframe that the data for the baseline weeks
        mental_data_avg:      The dataframe with the averages for the baseline and vegan weeks
        vegan_data_avg:       The dataframe with the averages for the vegan weeks
        baseline_data_avg:    The dataframe with the averages for the baseline weeks
        mental_weekly_avg:    The dataframes with the weekly averages for the baseline and vegan weeks
    
    """
    
    # reform subject to fit the overall structure; lowercase
    subject = subject.lower()
    
    # give the path where the needed files are located; with the function get_config
    config = get_config()
    mental_state_files_path = (config['mental_state_files_path'])

    # give the path where the processed files are located
    processed_files_path = mental_state_files_path + '/processed'

    # create the mental file
    mental_filename = processed_files_path + f'/mental_{subject}.csv'

    # create the baseline file
    baseline_filename = processed_files_path + f'/baseline_{subject}.csv'

    # create the vegan file
    vegan_filename = processed_files_path + f'/vegan_{subject}.csv'
    
    # create the mental average file
    mental_avg_filename = processed_files_path + f'/mental_{subject}_mean.csv'

    # create the baseline average file
    baseline_avg_filename = processed_files_path + f'/baseline_{subject}_mean.csv'

    # create the vegan average file
    vegan_avg_filename = processed_files_path + f'/vegan_{subject}_mean.csv'
    
    # create the vegan weekly average file
    mental_weekly_avg_filename = processed_files_path + f'/vegan_{subject}_weekly_mean.csv'
       
    # create dataframes for the whole period (mental), baseline and vegan period
    mental_data = pd.read_csv(mental_filename, sep=',')
    baseline_data = pd.read_csv(baseline_filename, sep=',')
    vegan_data = pd.read_csv(vegan_filename, sep=',')

    # create dataframes for the averages data; mental, baseline and vegan
    mental_data_avg = pd.read_csv(mental_avg_filename, sep=',')
    baseline_data_avg = pd.read_csv(baseline_avg_filename, sep=',')
    vegan_data_avg = pd.read_csv(vegan_avg_filename, sep=',')
    
    # create dataframes for the weekly averages
    mental_weekly_avg = pd.read_csv(mental_weekly_avg_filename, sep=',')
    
    # set 'date' column to datetime 
    mental_data['date'] = pd.to_datetime(mental_data['date'], format='%Y%m%d')
    baseline_data['date'] = pd.to_datetime(baseline_data['date'], format='%Y%m%d')
    vegan_data['date'] = pd.to_datetime(vegan_data['date'], format='%Y%m%d')
    
    # return all the created dataframes
    return mental_data, baseline_data, vegan_data, mental_data_avg, baseline_data_avg, vegan_data_avg, mental_weekly_avg

In [147]:
def mental_line_plot(subject, mental_data):
    """
    function returns a line plot for the whole 6 weeks including baseline and vegan period
    
    Parameters
        subject:            The subject's initial
        mental_data:        The dataframe with the whole 6 weeks of mental health data
    
    Returns
        line plot for the given subject for the baseline and vegan weeks with a separation line at the point where
        the diet was switched.
    
    """
    # give source for line plot
    source = ColumnDataSource(data=mental_data)
    
    # create separate dataframe with only numeric data which is to plot
    numeric_data = mental_data.drop(['current companion', 'current activity', 'location', 'notes', 'subject', 'usage'], axis=1)

    # create empty list where all columns are to be added, set index to 0
    options = []
    index = 0

    # loop through columns and add to list of column names; skip the first column as this is 'date'
    for (columnName, columnData) in numeric_data.iteritems():
        if index != 0:
            options.append(columnName)
        index = index + 1 

    # create dropdown menu to select which information top plot
    Axesselect = Select(title="Choose mood:", value="relaxed", options=options, css_classes=['selectDiv'])
    Axesselect.js_on_change('value', CustomJS(args=dict(source=source, Axesselect=Axesselect), code="""
        source.data['relaxed'] = source.data[Axesselect.value];
        source.change.emit();
    """))

    # create variable with the active axis
    controls = [Axesselect]
    
    # create the actual dropdown in a column
    inputs = column(*controls, width=300)

    # Create a plot
    p = figure(title=f'Subject {subject}:', plot_width=1000, plot_height=400, y_range=(0,7) , 
                x_axis_type='datetime', x_axis_label='Date', y_axis_label='Mood score')
    p.line('date', 'relaxed', source=source, color='#008ae0')

    # add tooltips to the plot to show the exact date and scores; as well as the textual data which cannot be plotted to
    # provive additional information.
    tooltips = HoverTool(
        tooltips = [("date", "@date{%F}"),
                    ('Score', '@relaxed'),
                    ('Notes', '@notes'),
                    ('Current companion', '@{current companion}'),
                    ('Usage', '@{usage}'),
                    ('Location', '@location')],
        formatters={'@date': 'datetime'}
    )
    
    # add the tooltips to the plot
    p.add_tools(tooltips)
    
    # set line at the border of baseline/vegan period
    border = time.mktime(datetime(2020, 10, 26, 3, 0, 0).timetuple())*1000
    daylight_savings_start = Span(location=border, dimension='height', line_color='#48BF91', line_width=3)
    p.add_layout(daylight_savings_start)

    # finish off by formatting the dropdown and plot
    final_layout = layout([
        [inputs],
        [p],
    ])
    
    # return dropdown and line plot
    return final_layout

In [188]:
def mental_bar_plot(subject, baseline_data_avg, vegan_data_avg):
    """
    Author: Henrike Vaartstra
     
    function returns a grouped bar plot with the averages of the baseline and vegan period
    
    Parameters
        subject:               The subject's initial
        baseline_data_avg:     Dataframe containing the average scores for the baseline weeks
        vegan_data_avg:        Dataframe containing the average scores for the vegan weeks
    
    Returns
        grouped bar plot for the given subject with the averages comparing baseline and vegan period
    
    """

    # merge the baseline and vegan dataframe for easy plotting and rename coluns
    vegan_baseline = pd.merge(baseline_data_avg, vegan_data_avg, on='mood')
    vegan_baseline = vegan_baseline.rename(columns={'score_x': 'baseline', 'score_y': 'vegan'})

    # create a list with all the unique moods in the dataframe
    moods = vegan_baseline['mood'].unique().tolist()

    # use columndatasource as source for the bars
    source = ColumnDataSource(vegan_baseline)

    # create figure
    p = figure(x_range=moods, y_range=(0, 7), title=f"Average moods baseline vs vegan for subject {subject}", plot_width=1000)

    # add the baseline bar
    p.vbar(x=dodge('mood', -0.23, range=p.x_range), top='baseline', width=0.4, source=source, 
           color="#008ae0", line_color="#007ecc", legend_label='Baseline')

    # add the vegan bar
    p.vbar(x=dodge('mood',  0.23,  range=p.x_range), top='vegan', width=0.4, source=source, 
           color="#48BF91", line_color="#3eb185", legend_label='Vegan')
    
    # add tooltips to see the scores shown in the bar
    p.add_tools(HoverTool(
        tooltips = [('Vegan', '@vegan'),
                   ('Baseline', '@baseline')]
    ))

    # add padding to the bars to be able to distinguish them
    p.x_range.range_padding = 0.05
    p.x_range.group_padding = 1.0
    
    # do not show grid spacing
    p.xgrid.grid_line_color = None
    
    # legend styling
    p.legend.location = "top_left"
    p.xaxis.major_label_orientation = 0.5
    p.legend.click_policy= 'hide'
    p.legend.orientation = "horizontal"

    # return trouped bar plot
    return p

In [149]:
def mental_radar_plot(subject, baseline_data_avg, vegan_data_avg):
    """
    function returns a radar plot comparing vegan and baseline

    Parameters
        subject:               The subject's initial
        baseline_data_avg:     The baseline data which is loaded in the function load_mental_files
        vegan_data_avg:        The vegan data which is loaded in the function load_mental_files
    
    Returns
        Radar plot with average scores during the baseline and the vegan period
    """

    # create a list with all the unique moods in the dataframe
    moods = baseline_data_avg['mood'].unique().tolist()
    
    # create emtpy figure
    p = go.Figure()

    # add radar trace with the baseline average scores
    p.add_trace(go.Scatterpolar(
        type='scatterpolar',
        r=baseline_data_avg['score'],
        theta = moods,
        fill='toself',
        line_color = '#007ecc',
        fillcolor = '#008ae0',
        opacity = 0.5,
        name='Baseline'
    ))
    
    # add radar trace with the vegan average scores
    p.add_trace(go.Scatterpolar(
        r=vegan_data_avg['score'],
        theta = moods,
        fill='toself',
        line_color = '#3eb185',
        fillcolor = '#48BF91',
        opacity = 0.5,
        name='Vegan'

    ))

    # update plot with the scale range and add title
    p.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[0, 7]
        )),
        showlegend=True,
        title = f'Total average mood scores of baseline vs. vegan for subject {subject}'
    )

    # return radar plot
    return p

In [168]:
def mental_radar_plot_weekly(subject, mental_weekly_avg):
    """
    function returns a radar plot per week showing the weekly averages
        
    Parameters
        subject:               The subject's initial
        mental_weekly_avg:     The baseline data which is loaded in the function load_mental_files

    Returns
        Radar plot with average scores during the baseline and the vegan period; shown per week
        
    """
    
    # create a list with all the unique moods in the dataframe
    moods = mental_weekly_avg['mood'].unique().tolist()

    # initiate empty tabs layout
    tabs = pn.Tabs()

    # skip the first column in weekly average as this is 'mood'
    columns = iter(mental_weekly_avg)
    next(columns)

    # loop through columns to plot every column in a separate radar plot
    for column in columns:
        # if baseline column; the radarplot gets the color designated to indicate baseline
        if 'baseline' in column:
            p = go.Figure(data=go.Scatterpolar(
                r=mental_weekly_avg[column],
                theta=moods,
                fill='toself',
                line_color = '#007ecc',
                fillcolor = '#008ae0',
                opacity = 0.5
            ))

        # else, a vegan column; the radarplot gets the color designated to indicate vegan    
        else:
            p = go.Figure(data=go.Scatterpolar(
                r=mental_weekly_avg[column],
                theta=moods,
                fill='toself',
                line_color = '#3eb185',
                fillcolor = '#48BF91',
                opacity = 0.5
            ))

        # update the layout by giving it the scale rating and title
        p.update_layout(
          polar=dict(
            radialaxis=dict(
              visible=True,
                range=[0,7]
            ),
          ),
            showlegend=False,
            title = f'Average mood scores of {column} for subject {subject}'
        )
        
        # append every plot to a tabular layout; every plot gets its own tab
        tabs.append((column, p))

    # return tabs layout
    return tabs

In [154]:
def mental_combine_radar(subject, mental_weekly_avg, baseline_data_avg, vegan_data_avg):
    """
    function combines radar_plot function and radar_weekly_plot 
    
    Parameters
        subject:            The subject's initial
        baseline_data:            The baseline data which is loaded in the function load_mental_files
        vegan_data:               The vegan data which is loaded in the function load_mental_files
    
    Returns
        Returns a tabular figure where to choose between total average or weekly
    """
    
    # assign weekly averages plot including tabs to a variable 'weekly'
    weekly = mental_radar_plot_weekly(subject, mental_weekly_avg)

    # assign total averages plot including tabs to a variable called 'averages'
    average = mental_radar_plot(subject, baseline_data_avg, vegan_data_avg)
    
    # assign both radar plots, including their own tabs, to a new tabular layout
    tabs = pn.Tabs(('Weekly average', weekly), ('Total average', average))

    # return total tabular layout
    return tabs

In [172]:
def create_mental_dataframe(mental_data):
    '''
    Author: Henrike Vaartstra
    
    Function creates the 'DOM' mental health dataframe for visual representation
    
    Parameters
        mental_data           The given DataFrame of the subject
    
    Returns
        panel DataFrame       The DataFrame widget created for visual representation
    '''
    
    # create the dataframe to show on panel; drop subject column and set date to date format to drop time
    mental_data_view = mental_data.drop(columns=['subject'])
    mental_data_view['date'] = mental_data_view['date'].dt.date
    mental_data_view = mental_data_view.set_index('date') 
    
    # create dataframe with the average scores
    mean = mental_data_view.mean(axis=0).round(2).to_frame('Mean').T
    
    # create dataframe with the median scores
    median = mental_data_view.median(axis=0).round(2).to_frame('Median').T

    # concatonate the mean and median dataframe to the original dataframe
    mental_data_df = pd.concat([mental_data_view, mean, median])
    
    # description of the mental health dataframe
    descr = Div(text="""
        <p><b>Mental state table</b><br>
        The mental state table shows the scores and answers given on the daily questionnaire. 
        The two bottom rows represent the mean and the median of the questions that had a 
        numeric scale answers.</p>
    """, width=600, height=70, css_classes=['plotDescription'])
    
    # set dataframe to a widget and freeze last two rows to set them at the bottom of the dataframe
    mental_df = pn.widgets.DataFrame(mental_data_df, autosize_mode='fit_columns', frozen_rows=-2,
                                width=1900, height=350)
    
    # return column with description and dataframe
    return pn.Column(descr, mental_df)

In [156]:
def create_nutrition_graphs(self):
    '''
    Retrieve all the food intake plots
    '''
    
    subjects = self.subjects
    figures = {}
    intake_dfs = {}
    for subject in subjects:
        nutrition_data = create_nutrition_tables(self, subject)
        averages_figure = create_average_nutrition_figure(subject, nutrition_data)
        boxplots = create_nutrition_boxplot(subject, nutrition_data)
        pie_charts = create_nutrition_piecharts(subject, nutrition_data)
        sub_figures = [boxplots, averages_figure, pie_charts]
        figures[subject] = sub_figures
        intake_dfs[subject] = create_nutrition_dataframe(nutrition_data)
    return figures, intake_dfs

In [157]:
def create_mental_state_graphs(self):
    '''
    Function that creates and returns the nutrition data graphs
    '''
    
    # set subjects, figures and mental_dfs
    subjects = self.subjects
    figures = {}
    mental_dfs = {}
    
    # loop through subjects to create all dataframes for every subject
    for subject in subjects:
        # every variable is assigned to a dataframe which is returned in the function load_mental_files
        mental_data = load_mental_files(subject)[0]
        baseline_data = load_mental_files(subject)[1]
        vegan_data = load_mental_files(subject)[2]
    
        mental_data_avg = load_mental_files(subject)[3]
        baseline_data_avg = load_mental_files(subject)[4]
        vegan_data_avg = load_mental_files(subject)[5]
        
        mental_weekly_avg = load_mental_files(subject)[6]
        
        # create plots
        line_plot_var = mental_line_plot(subject, mental_data)
        bar_plot_var = mental_bar_plot(subject, baseline_data_avg, vegan_data_avg)
        radar_plot_var = mental_combine_radar(subject, mental_weekly_avg, baseline_data_avg, vegan_data_avg)
        
        # create sub figures including all plots
        sub_figures = [line_plot_var, bar_plot_var, radar_plot_var]
        figures[subject] = sub_figures
        
        # create dataframe to show     
        mental_dfs[subject] = create_mental_dataframe(mental_data)
        
    # return all figures and dataframes   
    return figures, mental_dfs

In [158]:
def create_body_comp_graphs(self, subjects):
    '''
    Retrieve all body composition plots
    '''
    
    figures = {}
    body_comp_dfs = {}
    for subject in subjects:
        figures[subject] = create_body_comp_plots(self, subject)
        body_comp_dfs[subject] = create_body_comp_tables(self, subject)
    return figures, body_comp_dfs

In [159]:
def create_microbiota_graphs(self):
    '''
    Author: Tijs van Lieshout
    Retrieve all microbiota plots
    '''
    
    filepath = self.config['microbiota_files_path']
    gfkb, tax_profiles = load_data_microbiota(filepath)

    figures = {}
    
    dumbbell_plot = plot_dumbbell(tax_profiles, "s Pooled")
    zimmer_plot = recreate_zimmer(tax_profiles, "s Pooled")
    
    subjects = self.subjects
    for subject in subjects:
        dumbbell_plot = plot_dumbbell(tax_profiles, subject)
        zimmer_plot = recreate_zimmer(tax_profiles, subject)
        sub_figures = [dumbbell_plot, zimmer_plot]
        figures[subject] = sub_figures
    return figures

In [160]:
def create_food_intake_options(self):
    '''
    Create plot options for food intake visualisations
    '''
    
    self.food_intake_options = ['Boxplots', 'Average', 'Weekly piecharts']
    self.intake_plot_options = pn.widgets.Select(name='Food intake data:', options=self.food_intake_options, 
                                                 value='Boxplots', css_classes=['selectDiv'])
    self.intake_plot_options.param.watch(self.select_plot, 'value')
    self.table_checkbox = pn.widgets.Checkbox(name='View nutrition details', css_classes=['inputElement'])
    self.table_checkbox.param.watch(self.handle_intake_tables, 'value')

In [161]:
def create_microbiota_options(self):
    '''
    Create plot options for microbiota visualisations
    '''
    
    self.micr_options = ['Dumbbell plot', 'Zimmer plot']
    self.micr_plot_options = pn.widgets.Select(name='Microbiota data:', options=self.micr_options, 
                                                 value='Dumbbell plot', css_classes=['selectDiv'])
    self.micr_plot_options.param.watch(self.select_plot, 'value')

In [162]:
def create_mental_state_options(self):
    '''
    Create plot options for mental state visualisations
    '''
    
    self.mental_options = ['Line plot', 'Barplot', 'Radar plot']
    self.mental_plot_options = pn.widgets.Select(name="Mental state data:", options=self.mental_options,
                                                value='Barplot', css_classes=['selectDiv'])
    self.mental_plot_options.param.watch(self.select_plot, 'value')
    self.mental_checkbox = pn.widgets.Checkbox(name='View mental state details', css_classes=['inputElement'])
    self.mental_checkbox.param.watch(self.handle_mental_tables, 'value')

In [163]:
def create_body_comp_options(self):
    '''
    Create plot options for body composition visualisations
    '''
    
    self.body_comp_options = ['Body composition line plot', 'Body composition boxplots', 
                              'Body composition violin plots']
    self.body_comp_plot_options = pn.widgets.Select(name="Body composition data:", options=self.body_comp_options,
                                                   value='Body composition line plot', css_classes=['selectDiv'])
    self.body_comp_plot_options.param.watch(self.select_plot, 'value')
    self.body_comp_checkbox = pn.widgets.Checkbox(name='View body composition details', css_classes=['inputElement'])
    self.body_comp_checkbox.param.watch(self.handle_body_comp_tables, 'value')

In [164]:
def create_plot_descriptions(self):
    '''
    Function that creates the different plot descriptions
    '''
    
    self.descr_food_intake = div = [Div(text="""
        <h1>Food intake</h1>
        <p><b>Food intake boxplots</b><br>
        Boxplots are a standardized way of displaying the distribution of data based on a five number summary 
        ('minimum', first quartile (Q1), median, third quartile (Q3), and 'maximum'). These boxplots are divided 
        into four categories: Calories (kcal), Macronutrients (g), Carbohydrates (g) and Minerals (g) to compare 
        the different nutrients.</p>
    """, width=600, height=170, css_classes=['plotDescription']),
                                   Div(text="""
        <h1>Food intake</h1>
        <p><b>Food intake averages</b><br>
        The food intake average barplot represents the average nutrient intake during the normal diet compared 
        to the vegan diet.</p>
    """, width=600, height=130, css_classes=['plotDescription']),
                                   Div(text="""
        <h1>Food intake</h1>
        <p><b>Food intake pie charts</b><br>
        The pieces of a pie chart are proportional to the fraction of the whole in each category. 
        In other words, each slice of the pie is relative to the size of that category in the group as a whole. 
        The entire 'pie' represents 100% of a whole, while the pie 'slices' represent portions of the whole. 
        These pie charts represent the division of nutrients per week.</p>
    """, width=600, height=170, css_classes=['plotDescription'])]
    
    self.descr_body_comp = [Div(text="""
        <h1>Body composition</h1>
        <p><b>Body composition line plot</b><br>
        [insert body composition description]</p>
    """, width=600, height=150, css_classes=['plotDescription']),
                              Div(text="""
        <h1>Body composition</h1>
        <p><b>Body composition boxplots</b><br>
        [insert body composition description]</p>
    """, width=600, height=150, css_classes=['plotDescription']), 
                                 Div(text="""
        <h1>Body composition</h1>
        <p><b>Body composition violin plots</b><br>
        [insert body composition description]</p>
    """, width=600, height=150, css_classes=['plotDescription'])]
    
    self.descr_mental_state = [Div(text="""
        <h1>Mental state</h1>
        <p><b>Barplot</b><br>
        The mental health average barplot represents the average scores of the mental health survey during 
        the normal diet compared to the vegan diet. Only the average scores of questions with a rating 
        scale as an answer are shown.</p>
    """, width=600, height=150, css_classes=['plotDescription']),
                              Div(text="""
        <h1>Mental state</h1>
        <p><b>Line plot</b><br>
        The mental health line plot shows the daily scores of the chosen mood. The date on which the switch 
        was made from a normal diet to a vegan diet is indicated by the green line. Additional information such 
        as notes and company, which was filled in in the survey, can be seen when hovering over the line.</p>
    """, width=600, height=150, css_classes=['plotDescription']),
                              Div(text="""
        <h1>Mental state</h1>
        <p><b>Radar plot</b><br>
        The mental health radar plots represents the weekly and total averages of the mental health scores. 
        The radar plot is divided into two tabular plot where you can choose to see the weekly averages or the 
        total averages. The total averages radar plot shows the difference between baseline and vegan. Both plots
        only show theaverage scores of questions with a rating scale as an answer.</p>
    """, width=600, height=150, css_classes=['plotDescription'])]
    
    self.descr_microbiota = [Div(text="""
        <h1>Microbiota</h1>
        <p><b>Dumbbell plot</b><br>
        [insert dumbbell plot description]</p>
    """, width=600, height=150, css_classes=['plotDescription']), 
                            Div(text="""
        <h1>Microbiota</h1>
        <p><b>Zimmer plot</b><br>
        [insert zimmer plot description]</p>
    """, width=600, height=150, css_classes=['plotDescription'])]

In [165]:
class Dashboard:
    '''
    Author: Kylie Keijzer
    
    The main Dashboard class that serves the application display and functionalities 
    '''
    
    def __init__(self):
        self.config = get_config()
        self.subjects = ['A', 'B', 'C', 'D', 'E']
        self.measurements = ['Food intake', 'Body composition', 'Mental state', 'Microbiota']
        self.panel_options = DotMap()
        self.create_panel()
    
    def create_panel(self):
        nutrition_data = create_nutrition_graphs(self)
        self.nutrition_figures = nutrition_data[0]
        self.nutrition_tables = nutrition_data[1]
        mental_data = create_mental_state_graphs(self)
        self.mental_state_graphs = mental_data[0]
        self.mental_state_tables = mental_data[1]
        body_comp_data = create_body_comp_graphs(self, self.subjects)
        self.body_comp_graphs = body_comp_data[0]
        self.body_comp_tables = body_comp_data[1]
        self.microbiota_data = create_microbiota_graphs(self)

        self.subjects_menu = pn.widgets.Select(name='Subject:', options=self.subjects, value='A',
                                              css_classes=['selectDiv'])
        self.subjects_menu.param.watch(self.select_subject, 'value')
        
        self.measurement_label = Div(text="<b>Measurements:</b>", css_classes=['checkboxLabel'])
        self.checkbox_group = pn.widgets.CheckBoxGroup(name='Measurement', options=self.measurements,
                                                      value=['Food intake'], css_classes=['inputElement'])
        self.checkbox_group.param.watch(self.toggle_plots, 'value')
        
        create_food_intake_options(self)
        create_microbiota_options(self)
        create_mental_state_options(self)
        create_body_comp_options(self)
        create_plot_descriptions(self)
        
        # create the options menu
        self.options_column = pn.Column(self.subjects_menu, self.measurement_label, self.checkbox_group, 
                                        self.intake_plot_options, self.table_checkbox, 
                                        margin=0, sizing_mode='stretch_both')

        # display subject A and food intake boxplot as default
        self.panel_options.subject = 'A'
        self.panel_options.intake_plot = pn.Column(self.descr_food_intake[0], 
                                                   self.nutrition_figures['A'][0], css_classes=['figureElement'])
        self.panel_options.food_intake_displayed = True
        self.panel_options.microbiota_displayed = False
        self.panel_options.body_comp_displayed = False
        self.panel_options.mental_state_displayed = False
        self.panel_options.current_plots = DotMap({'food_intake': 'Boxplots'})
        
        self.figure_column = pn.Column(self.panel_options.intake_plot)
    
        self.template = pn.template.MaterialTemplate(title='Project Ve-gang')
        self.template.sidebar.append(self.options_column)
        self.template.main.append(self.figure_column)
        pn.serve(self.template)
    
    def select_subject(self, event):
        '''
        Function that handles the selection of a different subject
        '''
        
        if event.new:
            self.panel_options.subject = event.new
            
            if self.panel_options.food_intake_displayed:
                self.select_plot(DotMap({'new': self.panel_options.current_plots.food_intake}))
            if self.panel_options.microbiota_displayed:
                self.select_plot(DotMap({'new': self.panel_options.current_plots.microbiota}))
            if self.panel_options.body_comp_displayed:
                self.select_plot(DotMap({'new': self.panel_options.current_plots.body_comp}))
            if self.panel_options.mental_state_displayed:
                self.select_plot(DotMap({'new': self.panel_options.current_plots.mental_state}))
        
    def select_plot(self, event):
        '''
        Function that handles the selection of a different plot type
        '''
        
        if event.new:
            subject = self.panel_options.subject
            
            if (self.panel_options.food_intake_displayed and 
                event.new in self.food_intake_options):
                if event.new == 'Boxplots':
                    index = 0
                    self.panel_options.current_plots.food_intake = 'Boxplots'
                    descr = self.descr_food_intake[0]
                elif event.new == 'Average':
                    index = 1
                    self.panel_options.current_plots.food_intake = 'Average'
                    descr = self.descr_food_intake[1]
                elif event.new == 'Weekly piecharts':
                    index = 2
                    self.panel_options.current_plots.food_intake = 'Weekly piecharts'
                    descr = self.descr_food_intake[2]
                
                if self.panel_options.intake_plot in self.figure_column:
                    self.figure_column.remove(self.panel_options.intake_plot)
                self.panel_options.intake_plot = pn.Column(descr, self.nutrition_figures[subject][index],
                                                          css_classes=['figureElement'])
                if self.table_checkbox.value:
                    self.handle_intake_tables(DotMap({'new': True}))
                # add the nutrition figure to the panel
                self.figure_column.append(self.panel_options.intake_plot)
            
            elif (self.panel_options.microbiota_displayed and 
                  event.new in self.micr_options):
                if event.new == 'Dumbbell plot':
                    index = 0
                    self.panel_options.current_plots.microbiota = 'Dumbbell plot'
                    descr = self.descr_microbiota[0]
                elif event.new == 'Zimmer plot':
                    index = 1
                    self.panel_options.current_plots.microbiota = 'Zimmer plot'
                    descr = self.descr_microbiota[1]
                
                if self.panel_options.micr_plot in self.figure_column:
                    self.figure_column.remove(self.panel_options.micr_plot)
                self.panel_options.micr_plot = pn.Column(descr, self.microbiota_data[subject][index],
                                                        css_classes=['figureElement'])
                # add the microbiota figure to the panel
                self.figure_column.append(self.panel_options.micr_plot)
                
            elif (self.panel_options.body_comp_displayed and
                  event.new in self.body_comp_options):
                if event.new == 'Body composition line plot':
                    index = 0
                    self.panel_options.current_plots.body_comp = 'Body composition line plot'
                    descr = self.descr_body_comp[0]
                elif event.new == 'Body composition boxplots':
                    index = 1
                    self.panel_options.current_plots.body_comp = 'Body composition boxplots'
                    descr = self.descr_body_comp[1]
                elif event.new == 'Body composition violin plots':
                    index = 2
                    self.panel_options.current_plots.body_comp = 'Body composition violin plots'
                    descr = self.descr_body_comp[2]
                    
                if self.panel_options.body_comp_data in self.figure_column:
                    self.figure_column.remove(self.panel_options.body_comp_data)
                self.panel_options.body_comp_data = pn.Column(descr, self.body_comp_graphs[subject][index], 
                                                              css_classes=['figureElement'])
                if self.body_comp_checkbox.value:
                    self.handle_body_comp_tables(DotMap({'new': True}))
                # add the body compositions figure to the panel
                self.figure_column.append(self.panel_options.body_comp_data)
                
            elif (self.panel_options.mental_state_displayed and 
                 event.new in self.mental_options):
                if event.new == 'Barplot':
                    index = 1
                    self.panel_options.current_plots.mental_state = 'Barplot'
                    descr = self.descr_mental_state[0]
                elif event.new == 'Line plot':
                    index = 0
                    self.panel_options.current_plots.mental_state = 'Line plot'
                    descr = self.descr_mental_state[1]
                elif event.new == 'Radar plot':
                    index = 2
                    self.panel_options.current_plots.mental_state = 'Radar plot'
                    descr = self.descr_mental_state[2]
                
                if self.panel_options.mental_state_plot in self.figure_column:
                    self.figure_column.remove(self.panel_options.mental_state_plot)
                self.panel_options.mental_state_plot = pn.Column(descr, self.mental_state_graphs[subject][index], 
                                                                 css_classes=['figureElement'])
                if self.mental_checkbox.value:
                    self.handle_mental_tables(DotMap({'new': True}))
                # add the mental state figure to the panel
                self.figure_column.append(self.panel_options.mental_state_plot)

    def handle_intake_tables(self, event):
        '''
        Toggle show the intake datatables 
        '''
        
        subject = self.panel_options.subject
        # checked
        if event.new:
            self.panel_options.intake_plot.append(self.nutrition_tables[subject])
        # unchecked
        else:
            self.panel_options.intake_plot.remove(self.nutrition_tables[subject])
    
    def handle_mental_tables(self, event):
        '''
        Toggle show the mental state datatables 
        '''
        
        subject = self.panel_options.subject
        # checked
        if event.new:
            self.panel_options.mental_state_plot.append(self.mental_state_tables[subject])
        # unchecked
        else:
            self.panel_options.mental_state_plot.remove(self.mental_state_tables[subject])
    
    def handle_body_comp_tables(self, event):
        '''
        Toggle show the body composition datatables 
        '''
        
        subject = self.panel_options.subject
        # checked
        if event.new:
            self.panel_options.body_comp_data.append(self.body_comp_tables[subject])
        # unchecked
        else:
            self.panel_options.body_comp_data.remove(self.body_comp_tables[subject])
    
    def toggle_plots(self, event):
        '''
        Function that handles the check/uncheck of a measurement plot
        '''
        
        subject = self.panel_options.subject
        
        if not 'Food intake' in event.new and self.panel_options.food_intake_displayed:
            self.panel_options.food_intake_displayed = False
            self.figure_column.remove(self.panel_options.intake_plot)
            self.options_column.remove(self.intake_plot_options)
            self.options_column.remove(self.table_checkbox)
            self.panel_options.current_plots.food_intake = 'none'
        elif 'Food intake' in event.new and not self.panel_options.food_intake_displayed:
            self.panel_options.food_intake_displayed = True
            self.options_column.append(self.intake_plot_options)
            self.options_column.append(self.table_checkbox)
            self.intake_plot_options.value = 'Boxplots'
            self.select_plot(DotMap({'new': 'Boxplots'}))
        
        if not 'Body composition' in event.new and self.panel_options.body_comp_displayed:
            self.panel_options.body_comp_displayed = False
            self.figure_column.remove(self.panel_options.body_comp_data)
            self.options_column.remove(self.body_comp_plot_options)
            self.options_column.remove(self.body_comp_checkbox)
            self.panel_options.current_plots.body_comp = 'none'
        elif 'Body composition' in event.new and not self.panel_options.body_comp_displayed:
            self.panel_options.body_comp_displayed = True
            self.options_column.append(self.body_comp_plot_options)
            self.options_column.append(self.body_comp_checkbox)
            self.body_comp_plot_options.value = 'Body composition line plot'
            self.select_plot(DotMap({'new': 'Body composition line plot'}))
        
        if not 'Mental state' in event.new and self.panel_options.mental_state_displayed:
            self.panel_options.mental_state_displayed = False
            self.figure_column.remove(self.panel_options.mental_state_plot)
            self.options_column.remove(self.mental_plot_options)
            self.options_column.remove(self.mental_checkbox)
            self.panel_options.current_plots.mental_state = 'none'
        elif 'Mental state' in event.new and not self.panel_options.mental_state_displayed:
            self.panel_options.mental_state_displayed = True
            self.options_column.append(self.mental_plot_options)
            self.options_column.append(self.mental_checkbox)
            self.mental_plot_options.value = 'Barplot'
            self.select_plot(DotMap({'new': 'Barplot'}))
        
        if not 'Microbiota' in event.new and self.panel_options.microbiota_displayed:
            self.panel_options.microbiota_displayed = False
            self.figure_column.remove(self.panel_options.micr_plot)
            self.options_column.remove(self.micr_plot_options)
            self.panel_options.current_plots.microbiota = 'none'
        elif 'Microbiota' in event.new and not self.panel_options.microbiota_displayed:
            self.panel_options.microbiota_displayed = True
            self.options_column.append(self.micr_plot_options)
            self.micr_plot_options.value = 'Dumbbell plot'
            self.select_plot(DotMap({'new': 'Dumbbell plot'}))

In [189]:
def main():
    pn.extension(raw_css=[get_styling()])
    Dashboard()

if __name__ == '__main__':
    main()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning:

Mean of empty slice



Launching server at http://localhost:60597
